# Configuration

Define settings for remote services, create `appsettings.Development.json` using `appsettings.json` as template.

You need to manually give yourself
- `Cognitive Services OpenAI Contributor` rights on the `Azure OpenAI` resource
- `Search Index Data Contributor` right on the `Azure AI Search` resource

if you not already have it.

Also ensure to execute `azd auth login` to have access to the Azure resources.

> This example is based on [Creative Writing Assistant: Working with Agents using Prompty (Python Implementation)](https://github.com/Azure-Samples/contoso-creative-writer/tree/main).

# Install and Import Required Packages
Install and import the necessary packages using NuGet.

In [19]:
// Install the necessary packages using NuGet
#r "nuget: Azure.Identity, 1.13.1"
#r "nuget: CsvHelper, 33.0.1"
#r "nuget: Microsoft.Extensions.Configuration, 9.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Binder, 9.0.0"
#r "nuget: Microsoft.Extensions.Configuration.UserSecrets, 9.0.0"
#r "nuget: Microsoft.Extensions.Configuration.EnvironmentVariables, 9.0.0"
#r "nuget: Microsoft.Extensions.Http.Resilience, 9.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.33.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.33.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.33.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Yaml, 1.33.0"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.33.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Connectors.AzureAISearch, 1.33.0-preview"
#r "nuget: Microsoft.SemanticKernel.Connectors.AzureOpenAI, 1.33.0"
#r "nuget: Microsoft.SemanticKernel.Connectors.InMemory, 1.33.0-preview"

// Import the necessary libraries
using System;
using System.Threading;
using System.Net;
using System.ComponentModel;
using System.Diagnostics;
using System.IO;
using System.Text.Json;
using System.Threading.Tasks;
using System.Globalization;
using Azure.Identity;
using CsvHelper;
using Microsoft.DotNet.Interactive;
using Microsoft.Extensions.Configuration;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.Extensions.VectorData;
using Microsoft.Extensions.Http.Resilience;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.Chat;
using Microsoft.SemanticKernel.Agents.History;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.AzureAISearch;
using Microsoft.SemanticKernel.Connectors.AzureOpenAI;
using Microsoft.SemanticKernel.Connectors.InMemory;
using Microsoft.SemanticKernel.Data;
using Microsoft.SemanticKernel.Embeddings;
using Microsoft.SemanticKernel.Plugins.Web.Bing;

Installed Packages Azure.Identity, 1.13.1 CsvHelper, 33.0.1 Microsoft.Extensions.Configuration, 9.0.0 Microsoft.Extensions.Configuration.Binder, 9.0.0 Microsoft.Extensions.Configuration.EnvironmentVariables, 9.0.0 Microsoft.Extensions.Configuration.UserSecrets, 9.0.0 Microsoft.Extensions.Http.Resilience, 9.0.0 Microsoft.SemanticKernel, 1.33.0 Microsoft.SemanticKernel.Agents.Core, 1.33.0-alpha Microsoft.SemanticKernel.Connectors.AzureAISearch, 1.33.0-preview Microsoft.SemanticKernel.Connectors.AzureOpenAI, 1.33.0 Microsoft.SemanticKernel.Connectors.InMemory, 1.33.0-preview Microsoft.SemanticKernel.Plugins.Core, 1.33.0-alpha Microsoft.SemanticKernel.Plugins.Web, 1.33.0-alpha Microsoft.SemanticKernel.Yaml, 1.33.0

# Create Kernel Builder
Create a Kernel builder instance.

In [22]:
// The Agent Framework is experimental and requires warning suppression
#pragma warning disable CA2007, IDE1006, SKEXP0050, SKEXP0001, SKEXP0110, SKEXP0010, OPENAI001

var configBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("./appsettings.json", optional: false)
    .AddJsonFile("./appsettings.Development.json", optional: false);   
var configuration = configBuilder.Build();

// Create an embedding generation service.
var textEmbeddingGeneration = new AzureOpenAITextEmbeddingGenerationService(
    configuration["EmbeddingModelDeployment"],
    configuration["Endpoint"],
    new AzureDeveloperCliCredential());

// Construct an InMemory vector store.
var vectorStore = new InMemoryVectorStore();
var collectionName = "products";

private sealed class ProductDataModel
{
    [VectorStoreRecordKey]
    public Guid? Key { get; set; }

    [VectorStoreRecordData]
    [TextSearchResultName]
    public string Name { get; set; }

    [VectorStoreRecordData]
    [TextSearchResultValue]
    public string Content { get; set; }

    [VectorStoreRecordVector(1536)]
    public ReadOnlyMemory<float> Embedding { get; set; }
}

// Get and create collection if it doesn't exist.
var recordCollection = vectorStore.GetCollection<Guid, ProductDataModel>(collectionName);
await recordCollection.CreateCollectionIfNotExistsAsync();

using (var reader = new StreamReader("./../data/products.csv"))
using (var csv = new CsvReader(reader, CultureInfo.InvariantCulture))
{
    var records = csv.GetRecords<dynamic>().ToList();

    foreach (var record in records)
    {
        var product = new ProductDataModel
        {
            Key = Guid.NewGuid(),
            Name = record.Name,
            Content = record.Content,
            Embedding = await textEmbeddingGeneration.GenerateEmbeddingAsync((string)record.Name + "" + (string)record.Content)
        };
        await recordCollection.UpsertAsync(product);
    }
}

IKernelBuilder builder = Microsoft.SemanticKernel.Kernel
    .CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        configuration["ChatModelDeployment"],
        configuration["Endpoint"],
        new AzureDeveloperCliCredential());

builder.Services.ConfigureHttpClientDefaults(c =>
{
    c.AddStandardResilienceHandler().Configure(o =>
    {
        o.TotalRequestTimeout.Timeout = TimeSpan.FromSeconds(120);
        o.AttemptTimeout.Timeout = TimeSpan.FromSeconds(60);
        o.CircuitBreaker.SamplingDuration = TimeSpan.FromSeconds(120);
    });
});

private sealed class FunctionInvocationFilter() : IFunctionInvocationFilter
{
    public async Task OnFunctionInvocationAsync(FunctionInvocationContext context, Func<FunctionInvocationContext, Task> next)
    {
        if (context.Function.PluginName == "SearchPlugin")
        {
            Console.WriteLine($"{context.Function.Name}:{JsonSerializer.Serialize(context.Arguments)}");
        }
        await next(context);
    }
}

builder.Services.AddSingleton<IFunctionInvocationFilter, FunctionInvocationFilter>();

Microsoft.SemanticKernel.Kernel kernel = builder.Build();

Microsoft.SemanticKernel.Kernel bingKernel = kernel.Clone();
var textSearch = new BingTextSearch(apiKey: configuration["BingAPIKey"]);
var searchPlugin = textSearch.CreateWithSearch("SearchPlugin");
bingKernel.Plugins.Add(searchPlugin);

Microsoft.SemanticKernel.Kernel vectorSearchKernel = kernel.Clone();
var vectorTextSearch = new VectorStoreTextSearch<ProductDataModel>(recordCollection, textEmbeddingGeneration);
var vectorSearchPlugin = vectorTextSearch.CreateWithGetTextSearchResults("SearchPlugin");
vectorSearchKernel.Plugins.Add(vectorSearchPlugin);

# Create Researcher and Marketing agents, execute them

In [23]:
// The Agent Framework is experimental and requires warning suppression
#pragma warning disable CA2007, IDE1006, SKEXP0001, SKEXP0110, OPENAI001

const string ResearcherName = "Researcher";
const string MarketingName = "Marketing";
const string WriterName = "Writer";
const string EditorName = "Editor";

string researcherYaml = File.ReadAllText("./../ChatApp.WebApi/Agents/Prompts/researcher.yaml");
ChatCompletionAgent researcherAgent =
    new(KernelFunctionYaml.ToPromptTemplateConfig(researcherYaml))
    {
        Name = ResearcherName,
        Kernel = bingKernel,
        Arguments =
            new KernelArguments(
                new AzureOpenAIPromptExecutionSettings() 
                { 
                    FunctionChoiceBehavior = FunctionChoiceBehavior.Auto() 
                })
    };

string marketingYaml = File.ReadAllText("./../ChatApp.WebApi/Agents/Prompts/marketing.yaml");
ChatCompletionAgent marketingAgent =
    new(KernelFunctionYaml.ToPromptTemplateConfig(marketingYaml))
    {
        Name = MarketingName,
        Kernel = vectorSearchKernel,
        Arguments =
            new KernelArguments(
                new AzureOpenAIPromptExecutionSettings() 
                { 
                    FunctionChoiceBehavior = FunctionChoiceBehavior.Auto() 
                })
    };

var researchContext = "Can you find the camping trends in 2024 and what folks are doing in this winter?";
var productContext = "Can you use a selection of tents and sleeping bags as context?";
var assignment = @"Write a fun and engaging article that includes the research and product information. 
                    The article should be between 600 and 800 words.
                    Make sure to cite sources in the article as you mention the research not at the end.";

StringBuilder sbResearchResults = new();
await foreach (ChatMessageContent response in researcherAgent.InvokeAsync(new ChatHistory(), new(){{ "research_context", researchContext } }))
{
    Console.WriteLine();
    Console.WriteLine($"{response.AuthorName.ToUpperInvariant()}:{Environment.NewLine}{response.Content}");
    sbResearchResults.AppendLine(response.Content);
}

StringBuilder sbProductResults = new();
await foreach (ChatMessageContent response in marketingAgent.InvokeAsync(new ChatHistory(), new(){{ "product_context", productContext } }))
{
    Console.WriteLine();
    Console.WriteLine($"{response.AuthorName.ToUpperInvariant()}:{Environment.NewLine}{response.Content}");
    sbProductResults.AppendLine(response.Content);
}

Search:{"query":"camping trends 2024","count":"3"}
Search:{"query":"winter camping activities 2024","count":"3"}

RESEARCHER:
## Camping Trends in 2024
The camping trends for 2024 highlight that camping is more than just a temporary escape into the wild—it signifies an active decision to appreciate, protect, and experience nature in its fullest. Here are some key trends:

1. **Eco-Friendly Camping**: More campers are opting for sustainable practices, including using eco-friendly camping gear and minimizing their environmental impact.
2. **Tech-Enhanced Camping**: The integration of technology in camping gear, like solar-powered equipment and portable wifi solutions, is increasing.
3. **Health and Wellness Camping**: There is a rising trend in camping experiences that focus on health and wellness, such as yoga retreats and wellness camps.
4. **Adventure Camping**: Activities like rock climbing, mountain biking, and water sports are becoming more popular, making camping an adventure-pack

# Creat Writer and Editor agents, put them into Group Chat

In [24]:
// The Agent Framework is experimental and requires warning suppression
#pragma warning disable CA2007, IDE1006, SKEXP0001, SKEXP0110, OPENAI001

private sealed class NoFeedbackLeftTerminationStrategy : TerminationStrategy
{
    // Terminate when the final message contains the term "Article accepted, no further rework necessary." - all done
    protected override Task<bool> ShouldAgentTerminateAsync(Agent agent, IReadOnlyList<ChatMessageContent> history, CancellationToken cancellationToken)
    {
        if(agent.Name != EditorName)
            return Task.FromResult(false);

        return Task.FromResult(history[history.Count - 1].Content?.Contains("Article accepted", StringComparison.OrdinalIgnoreCase) ?? false);
    }
}



string writerYaml = File.ReadAllText("./../ChatApp.WebApi/Agents/Prompts/writer.yaml");
ChatCompletionAgent writerAgent =
    new(KernelFunctionYaml.ToPromptTemplateConfig(writerYaml))
    {
        Name = WriterName,
        Kernel = kernel,
        Arguments = new()
    };

string editorYaml = File.ReadAllText("./../ChatApp.WebApi/Agents/Prompts/editor.yaml");
ChatCompletionAgent editorAgent =
    new(KernelFunctionYaml.ToPromptTemplateConfig(editorYaml))
    {
        Name = EditorName,
        Kernel = kernel,
    };

AgentGroupChat chat =
    new(writerAgent, editorAgent)
    {
        ExecutionSettings = new AgentGroupChatSettings
        {
            SelectionStrategy = new SequentialSelectionStrategy(){ InitialAgent = writerAgent },
            TerminationStrategy = new NoFeedbackLeftTerminationStrategy()
        }
    };

Console.WriteLine("Ready!");

Ready!


# Execute Group Chat

In [25]:
// The Agent Framework is experimental and requires warning suppression
#pragma warning disable CA2007, IDE1006, SKEXP0001, SKEXP0110, OPENAI001

writerAgent.Arguments["research_context"] = researchContext;
writerAgent.Arguments["research_results"] = sbResearchResults.ToString();
writerAgent.Arguments["product_context"] = productContext;
writerAgent.Arguments["product_results"] = sbProductResults.ToString();
writerAgent.Arguments["assignment"] = assignment;

try
{
    await foreach (ChatMessageContent response in chat.InvokeAsync())
    {
        Console.WriteLine();
        Console.WriteLine("------------------------");
        Console.WriteLine($"{response.AuthorName.ToUpperInvariant()}:{Environment.NewLine}{response.Content}");
    }
}
catch (HttpOperationException exception)
{
    Console.WriteLine(exception.Message);
    if (exception.InnerException != null)
    {
        Console.WriteLine(exception.InnerException.Message);
        if (exception.InnerException.Data.Count > 0)
        {
            Console.WriteLine(JsonSerializer.Serialize(exception.InnerException.Data, new JsonSerializerOptions() { WriteIndented = true }));
        }
    }
}


------------------------
WRITER:
# The Ultimate Camping Guide: Trends and Gear for 2024

Are you an avid camper or just getting started on your outdoor adventures? Either way, 2024 brings exciting new camping trends that blend sustainability, technology, and a touch of luxury. Let's take a look at what this year's camping scene has to offer and the top gear to make your experience unforgettable.

## Camping Trends in 2024

### 1. Eco-Friendly Camping

In 2024, eco-friendly camping is more than a trend—it's a lifestyle choice. Campers are increasingly mindful of their environmental impact, opting for sustainable practices like using solar-powered gear, reusable water bottles, and biodegradable products. Brands are responding with eco-friendly options such as the Alpine Explorer Tent from AlpineGear, featuring breathable mesh windows and adjustable vents, all designed to work harmoniously with nature.

### 2. Tech-Enhanced Camping

Technology is making its way into the wilderness, chang